In [1]:
import pandas as pd
import numpy as np
import requests
import cfbd
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#home_id, away_id

In [3]:
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'RRRpRRQDqVoLX6S9Oivl7G9fXmZ/4TDcL9q+xg6pBW+yWfU1PIrYm/cbwKVuIfwn'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
try:
    # Games
    games = api_instance.get_games(year=2021)
    pprint(games[0])
except ApiException as e:
    print("Exception when calling BettingApi->get_lines: %s\n" % e)

{'attendance': 41064,
 'away_conference': 'Big Ten',
 'away_division': 'fbs',
 'away_id': 158,
 'away_line_scores': [0, 9, 7, 6],
 'away_points': 22,
 'away_post_win_prob': 0.5943393613382966,
 'away_postgame_elo': 1484,
 'away_pregame_elo': 1503,
 'away_team': 'Nebraska',
 'conference_game': True,
 'excitement_index': 5.4859812562,
 'highlights': None,
 'home_conference': 'Big Ten',
 'home_division': 'fbs',
 'home_id': 356,
 'home_line_scores': [2, 14, 14, 0],
 'home_points': 30,
 'home_post_win_prob': 0.4056606386617034,
 'home_postgame_elo': 1411,
 'home_pregame_elo': 1392,
 'home_team': 'Illinois',
 'id': 401282714,
 'neutral_site': False,
 'notes': None,
 'season': 2021,
 'season_type': 'regular',
 'start_date': '2021-08-28T17:20:00.000Z',
 'start_time_tbd': False,
 'venue': 'Memorial Stadium',
 'venue_id': 3832,
 'week': 1}


In [4]:
df = pd.DataFrame.from_records([dict(game_id=g.id,week=g.week,home_team=g.home_team, home_team_id=g.home_id, home_points=g.home_points, home_points_q=g.home_line_scores,away_points_q=g.away_line_scores,away_team_id=g.away_id, away_team=g.away_team, away_points=g.away_points,date=g.start_date,venue_id=g.venue_id,venue=g.venue,neutral_site=g.neutral_site) for g in games])
df.head()

,game_id,week,home_team,home_team_id,home_points,home_points_q,away_points_q,away_team_id,away_team,away_points,date,venue_id,venue,neutral_site
0,401282714,1,Illinois,356,30,"[2, 14, 14, 0]","[0, 9, 7, 6]",158,Nebraska,22,2021-08-28T17:20:00.000Z,3832.0,Memorial Stadium,False
1,401286187,1,Fresno State,278,45,"[7, 24, 7, 7]","[0, 0, 0, 0]",41,Connecticut,0,2021-08-28T18:00:00.000Z,3660.0,Bulldog Stadium,False
2,401329133,1,Central State,2119,6,"[0, 0, 0, 6]","[6, 7, 0, 7]",2310,Kentucky State,20,2021-08-28T19:00:00.000Z,3861.0,Ohio Stadium,True
3,401309833,1,UCLA,26,44,"[24, 7, 13, 0]","[3, 0, 7, 0]",62,Hawai'i,10,2021-08-28T19:30:00.000Z,1056.0,Rose Bowl,False
4,401328337,1,Indiana State,282,26,"[7, 3, 7, 9]","[0, 7, 7, 7]",2197,Eastern Illinois,21,2021-08-28T22:00:00.000Z,3829.0,Memorial Stadium,False


In [5]:
df = df[df['week'] < 13].copy()

In [6]:
home_df = df[['game_id','home_team_id','home_team','home_points','home_points_q','date','venue_id','venue','neutral_site','away_team','away_points','away_points_q','away_team_id']]
home_df.rename(columns={'away_team':'opponent','away_points':'points_allowed','away_points_q':'opponent_points_q','away_team_id':'opponent_id'},inplace=True)
home_points_split = home_df[['game_id','home_points_q']].copy()

/Users/kylekidder/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [7]:
home_points_split = home_df[['game_id','home_points_q']].copy()

In [8]:
home_points_split.dtypes

game_id           int64
home_points_q    object
dtype: object

In [9]:
split_df_home = pd.DataFrame(home_points_split['home_points_q'].to_list())

In [10]:
split_df_home.rename(columns={0:'home_points_q1',1:'home_points_q2',2:'home_points_q3',3:'home_points_q4'},inplace=True)
split_df_home = split_df_home[['home_points_q1','home_points_q2','home_points_q3','home_points_q4']].copy()

In [11]:
opponent_points_split = home_df[['game_id','opponent_points_q']].copy()

In [12]:
split_df_opponent = pd.DataFrame(opponent_points_split['opponent_points_q'].to_list())

In [13]:
split_df_opponent.rename(columns={0:'points_allowed_q1',1:'points_allowed_q2',2:'points_allowed_q3',3:'points_allowed_q4'},inplace=True)
split_df_opponent = split_df_opponent[['points_allowed_q1','points_allowed_q2','points_allowed_q3','points_allowed_q4']].copy()

In [14]:
split_df_home.head()

,home_points_q1,home_points_q2,home_points_q3,home_points_q4
0,2.0,14.0,14.0,0.0
1,7.0,24.0,7.0,7.0
2,0.0,0.0,0.0,6.0
3,24.0,7.0,13.0,0.0
4,7.0,3.0,7.0,9.0


In [15]:
split_df_opponent.head()

,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4
0,0.0,9.0,7.0,6.0
1,0.0,0.0,0.0,0.0
2,6.0,7.0,0.0,7.0
3,3.0,0.0,7.0,0.0
4,0.0,7.0,7.0,7.0


In [16]:
home_info = pd.concat([home_df,split_df_home,split_df_opponent],axis=1)
home_info['is_home'] = 1
home_info.rename(columns={'home_team_id':'team_id','home_points':'points','home_points_q1':'points_q1','home_points_q2':'points_q2','home_points_q3':'points_q3','home_points_q4':'points_q4','home_team':'school'},inplace=True)
home_info.drop(columns=['home_points_q','opponent_points_q'],axis=1,inplace=True)
home_info.head()

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
0,401282714,356,Illinois,30,2021-08-28T17:20:00.000Z,3832.0,Memorial Stadium,False,Nebraska,22,158,2.0,14.0,14.0,0.0,0.0,9.0,7.0,6.0,1
1,401286187,278,Fresno State,45,2021-08-28T18:00:00.000Z,3660.0,Bulldog Stadium,False,Connecticut,0,41,7.0,24.0,7.0,7.0,0.0,0.0,0.0,0.0,1
2,401329133,2119,Central State,6,2021-08-28T19:00:00.000Z,3861.0,Ohio Stadium,True,Kentucky State,20,2310,0.0,0.0,0.0,6.0,6.0,7.0,0.0,7.0,1
3,401309833,26,UCLA,44,2021-08-28T19:30:00.000Z,1056.0,Rose Bowl,False,Hawai'i,10,62,24.0,7.0,13.0,0.0,3.0,0.0,7.0,0.0,1
4,401328337,282,Indiana State,26,2021-08-28T22:00:00.000Z,3829.0,Memorial Stadium,False,Eastern Illinois,21,2197,7.0,3.0,7.0,9.0,0.0,7.0,7.0,7.0,1


In [17]:
################################################################

In [18]:
away_df = df[['game_id','away_team_id','away_team','away_points','away_points_q','date','venue_id','venue','neutral_site','home_team','home_points','home_team_id']]

In [19]:
away_points_split = away_df[['game_id','away_points_q']].copy()

In [20]:
split_df_away = pd.DataFrame(away_points_split['away_points_q'].to_list())

In [21]:
split_df_away.rename(columns={0:'away_points_q1',1:'away_points_q2',2:'away_points_q3',3:'away_points_q4'},inplace=True)
split_df_away = split_df_away[['away_points_q1','away_points_q2','away_points_q3','away_points_q4']].copy()

In [22]:
home_opponent_scores = split_df_home.copy()
home_opponent_scores.rename(columns={'home_points_q1':'points_allowed_q1','home_points_q2':'points_allowed_q2','home_points_q3':'points_allowed_q3','home_points_q4':'points_allowed_q4'},inplace=True)

In [23]:
away_info = pd.concat([away_df,split_df_away,home_opponent_scores],axis=1)
away_info['is_home'] = 0
away_info.rename(columns={'away_team_id':'team_id','away_points':'points','away_points_q1':'points_q1','away_points_q2':'points_q2','away_points_q3':'points_q3','away_points_q4':'points_q4','away_team':'school','home_points':'points_allowed','home_team':'opponent','home_team_id':'opponent_id'},inplace=True)
away_info.drop(columns=['away_points_q'],axis=1,inplace=True)
away_info.head()

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
0,401282714,158,Nebraska,22,2021-08-28T17:20:00.000Z,3832.0,Memorial Stadium,False,Illinois,30,356,0.0,9.0,7.0,6.0,2.0,14.0,14.0,0.0,0
1,401286187,41,Connecticut,0,2021-08-28T18:00:00.000Z,3660.0,Bulldog Stadium,False,Fresno State,45,278,0.0,0.0,0.0,0.0,7.0,24.0,7.0,7.0,0
2,401329133,2310,Kentucky State,20,2021-08-28T19:00:00.000Z,3861.0,Ohio Stadium,True,Central State,6,2119,6.0,7.0,0.0,7.0,0.0,0.0,0.0,6.0,0
3,401309833,62,Hawai'i,10,2021-08-28T19:30:00.000Z,1056.0,Rose Bowl,False,UCLA,44,26,3.0,0.0,7.0,0.0,24.0,7.0,13.0,0.0,0
4,401328337,2197,Eastern Illinois,21,2021-08-28T22:00:00.000Z,3829.0,Memorial Stadium,False,Indiana State,26,282,0.0,7.0,7.0,7.0,7.0,3.0,7.0,9.0,0


In [24]:
game_scoring_2021 = pd.concat([home_info,away_info],axis=0)

In [25]:
game_scoring_2021[game_scoring_2021['school']=='Alabama']

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
302,401282056,333,Alabama,48,2021-09-11T20:00:00.000Z,3657.0,Bryant Denny Stadium,False,Mercer,14,2382,14.0,17.0,14.0,3.0,0.0,0.0,7.0,7.0,1
779,401282081,333,Alabama,63,2021-09-25T23:30:00.000Z,3657.0,Bryant Denny Stadium,False,Southern Mississippi,14,2572,21.0,21.0,7.0,14.0,0.0,7.0,0.0,7.0,1
913,401282089,333,Alabama,42,2021-10-02T19:30:00.000Z,3657.0,Bryant Denny Stadium,False,Ole Miss,21,145,7.0,21.0,7.0,7.0,0.0,0.0,7.0,14.0,1
1548,401282111,333,Alabama,52,2021-10-23T23:00:00.000Z,3657.0,Bryant Denny Stadium,False,Tennessee,24,2633,7.0,14.0,3.0,28.0,14.0,0.0,3.0,7.0,1
1940,401282120,333,Alabama,20,2021-11-06T23:00:00.000Z,3657.0,Bryant Denny Stadium,False,LSU,14,99,0.0,14.0,6.0,0.0,7.0,0.0,7.0,0.0,1
1993,401282127,333,Alabama,59,2021-11-13T17:00:00.000Z,3657.0,Bryant Denny Stadium,False,New Mexico State,3,166,14.0,35.0,0.0,10.0,3.0,0.0,0.0,0.0,1
2258,401282135,333,Alabama,42,2021-11-20T20:30:00.000Z,3657.0,Bryant Denny Stadium,False,Arkansas,35,8,3.0,21.0,10.0,8.0,0.0,14.0,7.0,14.0,1
126,401281942,333,Alabama,44,2021-09-04T19:30:00.000Z,5348.0,Mercedes-Benz Stadium,True,Miami,13,2390,10.0,17.0,14.0,3.0,0.0,3.0,10.0,0.0,0
502,401282071,333,Alabama,31,2021-09-18T19:30:00.000Z,3634.0,Ben Hill Griffin Stadium,False,Florida,29,57,21.0,0.0,7.0,3.0,3.0,6.0,14.0,6.0,0
1179,401282103,333,Alabama,38,2021-10-10T00:00:00.000Z,3795.0,Kyle Field,False,Texas A&M,41,245,7.0,3.0,14.0,14.0,17.0,7.0,7.0,10.0,0


In [26]:
game_scoring_2021.to_csv('game_scoring_2021.csv')